# Cris Working Notebook
___

# 1.10 Friday - Data Mapping

In [2]:
import pandas as pd

In [ ]:
train = pd.read_csv("train.csv")
labels = pd.read_csv("train_labels.csv")
specs = pd.read_csv("specs.csv")

In [ ]:
print(f"dataset: {len(train)}")
print(f"event_id: {len(train.event_id)}")
print(f"game_session: {len(train.game_session)}")
print(" ")
print(f"unique event_codes: {len(train.event_code.unique())}")
print(f"unique title: {len(train.title.unique())}")

In [ ]:
train.head()

In [ ]:
train[train.world == "MAGMAPEAK"].groupby(["type","title"])["world"].count()

In [ ]:
magma = train[train.world == "MAGMAPEAK"]

In [ ]:
print(f"{train.type.value_counts()}")
print(len(train.event_id))

In [ ]:
len(train.event_id.unique())

In [ ]:
len(train.title.unique())

In [ ]:
len(train.event_code.unique())

In [ ]:
train.event_code.value_counts()

In [ ]:
bdf = pd.merge(train,specs,on="event_id")

In [ ]:
df = bdf[["event_code","event_id","info"]].sample(100_000)

In [ ]:
df[df.event_code==4035]

In [ ]:
labels.head()

---

# 1.13 Monday - Hypothesis Testing

## Summary of Insights:
1. There's a Very Weak Negative Correlation between the no. of assessments taken by a user, and their average accuracy.
 - This tells me that the accuracy is not about the amount of times assessments are taken.
 - There are distinct low, average, high performers - why?

In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# big = pd.read_csv("big.csv")

In [ ]:
big.head()

## Are the of Assessment Taken and Average Accuracy Correlated to Each Other?

### Look at data

In [ ]:
print("Accuracy Measures:")
sorted(big.accuracy.unique())

In [ ]:
plt.hist(x="accuracy",data=big)
plt.show()

In [ ]:
print(f"no. of unique installation ids: {len(big.installation_id.unique())}")

### Trim data for analysis

In [ ]:
assessment = big.groupby("installation_id")["accuracy"].agg(["count","mean"])

In [ ]:
assessment.head()

In [ ]:
assessment.columns

In [ ]:
assessment.columns = ["no_assessment","ave_accuracy"]
assessment.head()

In [ ]:
assessment.describe()

In [ ]:
plt.figure(figsize=(10,7))
plt.scatter(x="no_assessment",y="ave_accuracy",data=assessment[assessment.no_assessment <= 280])

plt.title("Number of Assessments Taken and the corresponding Average Accuracy")
plt.xlabel("Assesment Taken")
plt.ylabel("Ave Accuracy")

It seems like there are distinct low, average, high performers, that the amount of assessments they take does not matter. From the visualization, it does not seem like there is a correlation between the total no. of assessments taken, and the average accuracy. Nevertheless, let's do some statistical test to prove this.

### Plan:
1. Take 10 random samples and get the pearson's r for the two variables, `no_assessment` and `ave_accuracy`.
    - Get 1084 observations per sample, which is 30% of the population (3614)
2. Get the mean of the pearson's r of these 5 random samples to see the strength of correlation.

In [ ]:
s1 = assessment.sample(frac=.20)
s2 = assessment.sample(frac=.30)
s3 = assessment.sample(frac=.40)
s4 = assessment.sample(frac=.50)
s5 = assessment.sample(frac=.60)

Check that these samples are random.

In [ ]:
s1.head()

In [ ]:
s2.head()

In [ ]:
print(f"shape of sample 3: {s3.shape}")
print(f"shape of sample 4: {s4.shape}")
print(f"shape of sample 5: {s5.shape}")

In [ ]:
r1, pval1 = stats.pearsonr(s1.no_assessment, s1.ave_accuracy)
r2, pval2 = stats.pearsonr(s2.no_assessment, s2.ave_accuracy)
r3, pval3 = stats.pearsonr(s3.no_assessment, s3.ave_accuracy)
r4, pval4 = stats.pearsonr(s4.no_assessment, s4.ave_accuracy)
r5, pval5 = stats.pearsonr(s5.no_assessment, s5.ave_accuracy)

In [ ]:
samples = [s1,s2,s3,s4,s5]
sample_size = [len(sample) for sample in samples]

In [ ]:
pearsons_r = [r1,r2,r3,r4,r5]
pvalue = [pval1,pval2, pval3, pval4, pval5]

In [ ]:
metric_table = pd.DataFrame({"sample":["s1","s2","s3","s4","s5"],"pearsons_r":pearsons_r,"pvalue":pvalue,"sample_size":sample_size})

In [ ]:
metric_table.head()

In [ ]:
print(f"Average Pearsons R: {metric_table.pearsons_r.mean()}")
print("Very Weak Negative Relationship")

## What's the relationship between the type of activity engage in and their accuracy score? Are the users who did all three activities (game, activity, clip) in combination have higher accuracy scores?

In [8]:
import pandas as pd
import numpy as np
import wrangle

In [2]:
# test_takers = wrangle.get_assessment_users(cache=True)

In [6]:
test_takers.head()

,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,world,num_correct,num_incorrect,accuracy,accuracy_group,date,hour,weekday
1538,27253bdc,34ba1a28d02ba8ba,2019-08-06 04:57:18.904000+00:00,"{""event_code"": 2000, ""event_count"": 1}",0006a69f,1,2000,0,Welcome to Lost Lagoon!,Clip,NONE,NaN,NaN,NaN,NaN,2019-08-06,4,Tuesday
1539,27253bdc,4b57c9a59474a1b9,2019-08-06 04:57:45.301000+00:00,"{""event_code"": 2000, ""event_count"": 1}",0006a69f,1,2000,0,Magma Peak - Level 1,Clip,MAGMAPEAK,NaN,NaN,NaN,NaN,2019-08-06,4,Tuesday
1540,77261ab5,2b9d5af79bcdb79f,2019-08-06 04:58:14.538000+00:00,"{""version"":""1.0"",""event_count"":1,""game_time"":0...",0006a69f,1,2000,0,Sandcastle Builder (Activity),Activity,MAGMAPEAK,NaN,NaN,NaN,NaN,2019-08-06,4,Tuesday
1541,b2dba42b,2b9d5af79bcdb79f,2019-08-06 04:58:14.615000+00:00,"{""description"":""Let's build a sandcastle! Firs...",0006a69f,2,3010,29,Sandcastle Builder (Activity),Activity,MAGMAPEAK,NaN,NaN,NaN,NaN,2019-08-06,4,Tuesday
1542,1325467d,2b9d5af79bcdb79f,2019-08-06 04:58:16.680000+00:00,"{""coordinates"":{""x"":273,""y"":650,""stage_width"":...",0006a69f,3,4070,2137,Sandcastle Builder (Activity),Activity,MAGMAPEAK,NaN,NaN,NaN,NaN,2019-08-06,4,Tuesday


In [5]:
activity = test_takers[["installation_id","type","accuracy"]]
activity.head()

,installation_id,type,accuracy
1538,0006a69f,Clip,NaN
1539,0006a69f,Clip,NaN
1540,0006a69f,Activity,NaN
1541,0006a69f,Activity,NaN
1542,0006a69f,Activity,NaN


In [10]:
activity["c"] = np.where(activity.type=="Clip", 1, 0)
activity["a"] = np.where(activity.type=="Activity", 1, 0)
activity["g"] = np.where(activity.type=="Game", 1, 0)

In [11]:
activity.head()

,installation_id,type,accuracy,c,a,g
1538,0006a69f,Clip,NaN,1,0,0
1539,0006a69f,Clip,NaN,1,0,0
1540,0006a69f,Activity,NaN,0,1,0
1541,0006a69f,Activity,NaN,0,1,0
1542,0006a69f,Activity,NaN,0,1,0


In [22]:
act_c = activity.groupby("installation_id").c.sum()
act_a = activity.groupby("installation_id").a.sum()
act_g = activity.groupby("installation_id").g.sum()

In [38]:
#check if index match
# all(act_a.index == act_c.index)
# all(act_a.index == act_g.index)

True

In [40]:
act_agg = pd.DataFrame(act_c)
act_agg = act_agg.join([act_a,act_g])

In [41]:
act_agg.head()

,c,a,g
installation_id,,,
0006a69f,37,1771,1732
0006c192,32,1206,643
00129856,1,786,0
001d0ed0,38,76,712
00225f67,15,291,586
